In [11]:
import os
import json
import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

RuntimeError: Inter op parallelism cannot be modified after initialization.

In [14]:
train_dir = r"C:\Users\KIIT\Desktop\MajorProject\AgriShield\data\processed\PlantVillageDataset\train_val_test\train"
val_dir   = r"C:\Users\KIIT\Desktop\MajorProject\AgriShield\data\processed\PlantVillageDataset\train_val_test\val"
test_dir  = r"C:\Users\KIIT\Desktop\MajorProject\AgriShield\data\processed\PlantVillageDataset\train_val_test\test"


In [15]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
EPOCHS = 25

NUM_CLASSES = len(os.listdir(train_dir))

print(f"\n⚙️ Configuration:")
print(f"  Image size: {IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Classes: {NUM_CLASSES}")


⚙️ Configuration:
  Image size: (160, 160)
  Batch size: 32
  Epochs: 25
  Classes: 15


In [16]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=12,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print(f"\n📊 Dataset Loaded:")
print(f"  Training images: {train_generator.samples}")
print(f"  Validation images: {val_generator.samples}")
print(f"  Test images: {test_generator.samples}")

Found 14437 images belonging to 15 classes.
Found 2073 images belonging to 15 classes.
Found 4128 images belonging to 15 classes.

📊 Dataset Loaded:
  Training images: 14437
  Validation images: 2073
  Test images: 4128


In [17]:
print("\n🚀 Loading MobileNetV2...")
base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(160, 160, 3),
    alpha=1.0
)

base_model.trainable = False  # Freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)


🚀 Loading MobileNetV2...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step


In [18]:
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("\n📋 Model Summary:")
model.summary()


📋 Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 160, 160, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 80, 80, 32)        │             864 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 80, 80, 32)        │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 80, 80, 32)        │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 80, 80, 32)        │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 80, 80, 32)        │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 80, 80, 32)        │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 80, 80, 16)        │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 80, 80, 16)        │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 80, 80, 96)        │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 80, 80, 96)        │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 80, 80, 96)        │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 81, 81, 96)        │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 40, 40, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,620,751 (10.00 MB)

 Trainable params: 362,767 (1.38 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [20]:
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        "best_cpu_model.keras",
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

In [21]:
print("\n" + "=" * 70)
print("PHASE 1: TRAINING (Frozen Base Model)")
print("=" * 70)

steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = val_generator.samples // BATCH_SIZE

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)


PHASE 1: TRAINING (Frozen Base Model)


C:\Users\KIIT\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4685 - loss: 1.6797
Epoch 1: val_accuracy improved from -inf to 0.82617, saving model to best_cpu_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 853s 2s/step - accuracy: 0.4689 - loss: 1.6787 - val_accuracy: 0.8262 - val_loss: 0.5634 - learning_rate: 0.0010
Epoch 2/25
  1/451 ━━━━━━━━━━━━━━━━━━━━ 2:43 364ms/step - accuracy: 0.6250 - loss: 1.0097

C:\Users\KIIT\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.82617 to 0.83008, saving model to best_cpu_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.6250 - loss: 1.0097 - val_accuracy: 0.8301 - val_loss: 0.5600 - learning_rate: 0.0010
Epoch 3/25
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 977ms/step - accuracy: 0.7544 - loss: 0.7446
Epoch 3: val_accuracy improved from 0.83008 to 0.84814, saving model to best_cpu_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - accuracy: 0.7544 - loss: 0.7446 - val_accuracy: 0.8481 - val_loss: 0.4655 - learning_rate: 0.0010
Epoch 4/25
  1/451 ━━━━━━━━━━━━━━━━━━━━ 2:31 336ms/step - accuracy: 0.8125 - loss: 0.6400
Epoch 4: val_accuracy did not improve from 0.84814
451/451 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - accuracy: 0.8125 - loss: 0.6400 - val_accuracy: 0.8481 - val_loss: 0.4675 - learning_rate: 0.0010
Epoch 5/25
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 865ms/step - accuracy: 0.7937 - loss: 0.6287
Epoch 5: val_accuracy improved from 0.84814 to 0.86816, saving model

In [22]:
print("\n📊 Evaluating on Test Set...")
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print("\n" + "=" * 70)
print("FINAL RESULTS")
print("=" * 70)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss:     {test_loss:.4f}")
print("=" * 70)


📊 Evaluating on Test Set...
129/129 ━━━━━━━━━━━━━━━━━━━━ 97s 753ms/step - accuracy: 0.9108 - loss: 0.2703

FINAL RESULTS
Test Accuracy: 89.27%
Test Loss:     0.3209


In [24]:
# Load the best model (saved by checkpoint)
best_model = tf.keras.models.load_model("best_cpu_model.keras")

print("\n" + "="*70)
print("EVALUATING ON TEST SET")
print("="*70)

# Evaluate on test set
test_loss, test_acc = best_model.evaluate(test_generator, verbose=1)

print("\n" + "="*70)
print("TEST RESULTS")
print("="*70)
print(f"Test Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss:      {test_loss:.4f}")
print("="*70)

# Compare with your old model performance
print("\nComparison with old model:")
print(f"  Old Model:  84.54%")
print(f"  New Model:  {test_acc*100:.2f}%")
print(f"  Improvement: {(test_acc*100 - 84.54):.2f}%")


EVALUATING ON TEST SET
129/129 ━━━━━━━━━━━━━━━━━━━━ 50s 358ms/step - accuracy: 0.9108 - loss: 0.2703

TEST RESULTS
Test Accuracy:  0.8927 (89.27%)
Test Loss:      0.3209

Comparison with old model:
  Old Model:  84.54%
  New Model:  89.27%
  Improvement: 4.73%


In [25]:
model.save("final_cpu_optimized_model.keras")
print("\n✅ Model saved as 'final_cpu_optimized_model.keras'")

with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f, indent=2)

print("✅ Class indices saved as 'class_indices.json'")


✅ Model saved as 'final_cpu_optimized_model.keras'
✅ Class indices saved as 'class_indices.json'
